In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.functions import to_date
df1=df.withColumn('review_date', to_date('review_date', 'yyyy-MM-dd'))
df1.show(5)

In [ ]:
df1.printSchema()

In [ ]:
df1.createTempView('mobile_apps')

In [ ]:
review_id_table= spark.sql('select review_id, customer_id, product_id, product_parent, review_date from mobile_apps')
review_id_table.show(5)

In [ ]:
products=spark.sql("select product_id,product_title from mobile_apps")
products.show(5)

In [ ]:
customer_df=spark.sql('select customer_id, count(customer_id) as customer from mobile_apps group by customer_id')
customer_df.show()

In [ ]:
vine_table=spark.sql('select review_id, star_rating, helpful_votes, total_votes, vine from mobile_apps')
vine_table.show(5)

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/mobile_apps"
config = {"user":"root",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)